 <img src="../images/logos/Ideam_logo.png" width=300 alt="Ideam_Logo"></img> 

# Estaciones hidrometeorológicas

---

## Introducción

En este cuadernillo (Notebook) aprenderemos:

1. Introduccion a la red de monitoreo del IDEAM
2. Cátalogo de estaciones de IDEAM
3. Consulta de datos usando la plataforma [datosabiertos.gov.co](https://www.datos.gov.co/) 
1. Consulta de datos de temperatura y precipitación
1. Otros datos disponibles

## Prerequisitos
| Conceptos | Importancia | Notas |
| --- | --- | --- |
| [Introducción a Pandas](https://foundations.projectpythia.org/core/pandas.html) | Necesario | Lectura de datos tabulares |
| [Introducción a Datetime](https://foundations.projectpythia.org/core/datetime/datetime.html) | Necesario | Entender estampas de tiempo |
| [Introducción a Cartopy](https://foundations.projectpythia.org/core/cartopy.html) | Necesario | Entender estampas de tiempo |
| [Introducción a folium](https://python-visualization.github.io/folium/latest/getting_started.html) | Útil | Mapas interactivos |


- **Tiempo de aprendizaje**: 30 minutos

## 1. Catálogo nacional de estaciones de IDEAM 

Según el catálogo de estaciones hidrometeorológicas del IDEAM, el país cuenta con alrededor de 4.400 estaciones de diferentes categorías. En el siguiente cuadro se resume el estado de las estaciones por categoría de acuerdo a la PQR No. 20229050190832 (Enero de 2023)

| Categoría | Activa | Mantenimiento | Suspendidas |
| --- | :---: | :---: | :---: |
| Limnográfica  |  287  | 109  | 106  |
| Climátologica principal |  215  |  60  |  92  |
| Mareográfica | 4  | 2  | 2  |
| Pluviográfica | 104  | 0  | 87  |
| Limnométrica | 323  | 11  | 557  |
| Climática Ordinaria | 211  | 31  | 253  |
| Agrometeorológica | 51  | 4  | 57  |
| Radio Sonda | 6  | 2  | 2  |
| Pluviométrica | 1109  | 9  | 603  |
| Meteorológica Especial | 40  | 4  | 68  |
| Sinóptica Principal | 27  | 3  | 4  |
| Sinóptica Secundaria | 2  | 0  | 5  |
| **Total** | **2381**  | **235** | **1866**  |

## Librerías

A continuación vamos a importar las librerías que utilizaremos en este cuadernillo.

In [ ]:
from datetime import datetime, timedelta

import cartopy.crs as ccrs
import cartopy.feature as feature
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.dates import DateFormatter, HourLocator
from pandas import to_datetime

## 2. Acceso al catálogo en [bart.ideam.gov.co](http://bart.ideam.gov.co/cneideam/) 

El catálogo nacional de estaciones del IDEAM actualizado se encuentra disponible en el servidor [Bart](http://bart.ideam.gov.co). Podemos leer el catálogo usando `pandas.read_excel` como se muestra a continuación:

In [ ]:
df_cat = pd.read_excel("http://bart.ideam.gov.co/cneideam/CNE_IDEAM.xls")

In [ ]:
# df_cat.head()

In [ ]:
df_cat.info()

### 2.1 Mapa de estaciones 

Podemos usar `cartopy` para hacer un mapa y visualizar las estaciones de monitoreo en el país

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()}, dpi=150)
ax.coastlines()
gl = ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree())
ax.scatter(df_cat["LONGITUD"], df_cat["LATITUD"], transform=ccrs.PlateCarree(), s=0.5)
ax.add_feature(feature.LAND)
ax.add_feature(feature.OCEAN)
ax.add_feature(feature.COASTLINE, linewidth=0.5)
ax.add_feature(feature.BORDERS, linewidth=0.5)

podemos agrupar la data por área operativa, tipo de estación, tecnología, y otras variables

In [ ]:
# df_grp = df_cat.groupby('AREA_OPERATIVA')
# df_grp = df_cat.groupby('TECNOLOGIA')
df_grp = df_cat.groupby("ESTADO")

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": ccrs.PlateCarree()}, dpi=150)

for _, group in df_grp:
    ax.scatter(
        group["LONGITUD"],
        group["LATITUD"],
        transform=ccrs.PlateCarree(),
        s=0.5,
        label=_,
    )

ax.coastlines()
gl = ax.gridlines(draw_labels=True, crs=ccrs.PlateCarree())
ax.add_feature(feature.LAND)
ax.add_feature(feature.OCEAN)
ax.add_feature(feature.COASTLINE, linewidth=0.5)
ax.add_feature(feature.BORDERS, linewidth=0.5)
ax.legend(fontsize=5)

Podemos validar el número total de estaciones activas, en matenimiento y suspendidas de acuerdo con la información contenida en el catálogo

In [ ]:
for grp in df_grp.groups.keys():
    print(f"{grp}: {len(df_grp.get_group(grp))}")

### 2.2 Mapa de estaciones interactivo

También podemos hacer mapas interactivos usando [folium](https://python-visualization.github.io/folium/latest/)

In [ ]:
import folium
from folium import plugins
from folium.plugins import MarkerCluster

In [ ]:
min_lon, max_lon, min_lat, max_lat = -90, -72, -1, 14

map_ = folium.Map(
    location=[8, -76],
    zoom_start=6,
    min_lat=min_lat,
    max_lat=max_lat,
    min_lon=min_lon,
    max_lon=max_lon,
    zoom_control=False,
    control_scale=True,
    scrollWheelZoom=True,
    width=1000,
    height=600,
)
marker_cluster = MarkerCluster(name="Estaciones").add_to(map_)

folium.TileLayer("cartodbpositron").add_to(map_)
folium.TileLayer("openstreetmap").add_to(map_)
folium.TileLayer("cartodbdark_matter").add_to(map_)
folium.LayerControl().add_to(map_)

minimap = plugins.MiniMap()
_ = map_.add_child(minimap)

Ahora agregamos las estaciones usando la siguiente función:

In [ ]:
def plot_station(row):
    """input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map"""

    html = row.to_frame("_").to_html(
        classes="table table-striped table-hover table-condensed table-responsive"
    )
    popup = folium.Popup(html, max_width=2650)
    folium.Marker(location=[row['LATITUD'], row['LONGITUD']], popup=popup).add_to(
        marker_cluster
    )

In [ ]:
df_cat.apply(plot_station, axis=1)
map_

## 3. Acceso a la información histórica de IDEAM usando [datosabiertos.gov.co](https://www.datos.gov.co/)

la información histórica de múltiples sensores se puede consultar a través de la plataforma de datos abiertos usando el aplicativo [sodapy](https://dev.socrata.com/). Socrata utiliza un módulo denominado `Socrata` que permite realizar consultas al repositorio. Cada variable hidrometeorógica dispuesta se puede consultar usando el su respectivo código del set de datos.


| Variable | Código del set de datos |
| --- | --- |
| [Dirección del viento](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Direcci-n-Viento/kiw7-v9ta) | kiw7-v9ta |
| [Nivel instantáneo](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Nivel-Instant-neo/bdmn-sqnh) | bdmn-sqnh |
| [Temperatura Mínima del Aire](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Temperatura-M-nima-del-Aire/afdg-3zpb) | afdg-3zpb |
| [Temperatura Máxima del Aire](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Temperatura-M-xima-del-Aire/ccvq-rp9s) | ccvq-rp9s |
| [Velocidad del Viento](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Velocidad-Viento/sgfv-3yp8) | sgfv-3yp8 |
| [Nivel Máximo](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Nivel-M-ximo/vfth-yucv) | vfth-yucv |
| [Nivel Mínimo](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Nivel-M-nimo/pt9a-aamx) | pt9a-aamx |
| [Humedad del Aire](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Humedad-del-Aire-2-metros/uext-mhny) | uext-mhny |
| [Temperatura](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Datos-Hidrometeorol-gicos-Crudos-Red-de-Estaciones/sbwg-7ju4) | sbwg-7ju4 |
| [Nivel del mar mínimo](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Nivel-del-Mar-M-nimo/7z6g-yx9q) | 7z6g-yx9q |
| [Nivel del mar máximo](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Nivel-del-Mar-M-ximo/uxy3-jchf) | uxy3-jchf |
| [Nivel del mar](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Nivel-del-Mar/ia8x-22em) | ia8x-22em |
| [Presión Atmosferica](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Presi-n-Atmosf-rica/62tk-nxj5) | 62tk-nxj5 |
| [Precipitación](https://www.datos.gov.co/Ambiente-y-Desarrollo-Sostenible/Precipitaci-n/s54a-sgyg) | s54a-sgyg |

In [ ]:
# importamos la libreria Socrata
from sodapy import Socrata

### 3.1 Precipitación (s54a-sgyg)

Vamos a consultar los datos de `precipitación` reportada en la página, por ende vamos a usar el código `s54a-sgyg`. Para esto usamos el método `Socrata`, pasamos la dirección del repositorio y `None` que corresponde a la no autenticación

In [ ]:
# conexión cliente usando socrata al repositorio de datos abiertos
client = Socrata("www.datos.gov.co", None)

Una vez creado el cliente empezamos a hacer la consulta de datos  usando `client.get` y pasando los respectivos parámetros `dataset_identifier`, de la tabla anterior , y `limit` para generar consultas no muy grandes para efectos demostrativos. El resultado es una lista con múltiples diccionarios como se puede ver a continuación.

In [ ]:
# Solicitud de informacion al repositorio de interés
results = client.get(dataset_identifier="s54a-sgyg", limit=2000)
results[:1]

Estos resultados los podemos convertir en un `Dataframe` usando [pandas.Dataframe.from_records](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.from_records.html)

In [ ]:
results_df = pd.DataFrame.from_records(results)
results_df.head()

Ahora podemos usar filtrar los datos por diferentes campos como el `codigoestacion`, `fechaobservacion`, o `valorobservado`. Podemos pasar parámetros `SQL` como `where`, `AND`, `IN`, entre otros, en el método `client.get`

In [ ]:
# client.get?

In [ ]:
# Solicitud de informacion para la estación de la Universidad Nacional - Bogotá - 0021205012
ppt_query = client.get(
    dataset_identifier="s54a-sgyg",
    select="fechaobservacion, valorobservado, codigoestacion",
    where="codigoestacion IN ('0021205012') \
                              AND fechaobservacion > '2017'",
)
df_est = pd.DataFrame.from_records(ppt_query)
df_est.head()

#### Gráfico de la serie temporal 
Podemos generar una serie temporal usando la información resultado de la consulta. Sin embargo, primero debemos revisar el tipo de dato de cada columna

In [ ]:
df_est.info()

In [ ]:
df_est["fechaobservacion"] = pd.to_datetime(df_est["fechaobservacion"])
df_est.set_index("fechaobservacion", inplace=True)
df_est.valorobservado = df_est["valorobservado"].astype(float)
df_est = df_est.sort_index()
df_est.tail()

In [ ]:
# pd.options.plotting.backend = 'holoviews'
fig, ax = plt.subplots(figsize=(12, 3))
df_est["valorobservado"].plot(ax=ax, drawstyle="steps")

Podemos solicitar información para estaciones que reportan datos en el último mes

In [ ]:
ppt_query = client.get(
    dataset_identifier="s54a-sgyg",
    select="fechaobservacion, valorobservado, codigoestacion, nombreestacion",
    where="fechaobservacion > '2023-09-11'",
)
df_ult = pd.DataFrame.from_records(ppt_query)
df_ult.head()

### 3.2 Temperatura (sbwg-7ju4)

De manera similar podemos consultar otros registros como los de temperatura. Cambiamos el identificador de set de datos y generamos una nueva consulta

In [ ]:
# Solicitud de informacion para la estación de la Universidad Nacional - Bogotá - 0021205012
temp_query = client.get(
    dataset_identifier="sbwg-7ju4",
    select="fechaobservacion, valorobservado, codigoestacion",
    where="codigoestacion IN ('0021205012') \
                              AND fechaobservacion > '2017'",
)
df_temp = pd.DataFrame.from_records(temp_query)
df_temp.index = pd.to_datetime(df_temp["fechaobservacion"])
df_temp.valorobservado = df_temp["valorobservado"].astype(float)
df_temp = df_temp.sort_index()
df_temp.tail()

In [ ]:
fig, ax = plt.subplots(figsize=(12, 3))
df_temp["valorobservado"].plot(c="C00", lw=0.5, ax=ax)

## 4. Datos en tiempo "Causi-real" de IDEAM 

De igual manera, el IDEAM dispone de una tabla de datos  en [tiempo cercano a la medición](https://www.datos.gov.co/es/Ambiente-y-Desarrollo-Sostenible/Datos-de-Estaciones-de-IDEAM-y-de-Terceros/57sv-p2fu). Esta tabla corresponde al `dataset_identifier="57sv-p2fu"`. Generemos una consulta básica para ver los campos contenidos dentro de esta tabla en el último día

In [ ]:
time_now = datetime.now()
time = time_now - timedelta(days=10)
time

Convertimos la fecha en un `str` para incluirlo en la consulta

In [ ]:
time_str = f"{to_datetime(time):%Y-%m-%d}"
time_str

In [ ]:
nrt_query = client.get(
    dataset_identifier="57sv-p2fu",
    select="*",
    where="fechaobservacion >= '{}'".format(time_str),
    limit=2000,
)
df_nrt = pd.DataFrame.from_records(nrt_query)

In [ ]:
df_nrt.head()

Los primeros registros nos indican que hay mediciones cercanas a las fechas de la ejecución de este cuadernillo. Generemos una consulta más específica para la estacion `0024035340` correspondiente al Aeropuerto Alberto Lleras Camargo de Sogamoso. 

In [ ]:
cod_est = "0024035340"

In [ ]:
aero_query = client.get(
    dataset_identifier="57sv-p2fu",
    select="*",
    where="fechaobservacion >= '{}'\
                        AND codigoestacion IN ('{}')".format(
        time_str, cod_est
    ),
    limit=2000,
)
df_aero = pd.DataFrame.from_records(aero_query)

In [ ]:
df_aero.head(10)

Para validar los sensores que tiene esta estacion podemo usar el método `.unique()` de `Pandas`

In [ ]:
df_aero["codigosensor"].unique()

Podemos centrar aún mas la consulta agregándole el sensor de temperatura `codigosensor=0071`

In [ ]:
cod_sensor = "0071"

In [ ]:
aero_query = client.get(
    dataset_identifier="57sv-p2fu",
    select="fechaobservacion, valorobservado",
    where="fechaobservacion >= '{}'\
                        AND codigoestacion IN ('{}') \
                        AND codigosensor IN ('{}')".format(
        time_str, cod_est, cod_sensor
    ),
    limit=2000,
)
df_aero = pd.DataFrame.from_records(aero_query)

In [ ]:
df_aero

Ahora generemos un gráfico rápido de la serie de temperatura para las últimas 24 horas

In [ ]:
fig, ax = plt.subplots(figsize=(10, 3))
df_aero.index = pd.to_datetime(df_aero["fechaobservacion"])
df_aero.valorobservado = df_aero["valorobservado"].astype(float)
df_aero.plot(ax=ax)
ax.xaxis.set_major_locator(HourLocator(interval=4))  # tick every four hours
ax.xaxis.set_major_formatter(DateFormatter("%m-%d\n%H:%M"))

---

## Conclusiones
En este cuadernillo aprendimos una manera fácil y rápida como acceder a la información histórica y presente de las estaciones hidrometeorológicas del IDEAM. De igual modo aprendimos a visualizar las estaciones usando mapas interactivos. También aprendimos a generar consultas a diferentes grupos de datos usando sintaxis SQL y el aplicativo `socrata` de la plataforma de datos abiertos del gobierno Colombiano. 

## Resources and references

* Rose, B. E. J., Kent, J., Tyle, K., Clyne, J., Banihirwe, A., Camron, D., May, R., Grover, M., Ford, R. R., Paul, K., Morley, J., Eroglu, O., Kailyn, L., & Zacharias, A. (2023). Pythia Foundations (Version v2023.05.01) https://doi.org/10.5281/zenodo.7884572